<a href="https://colab.research.google.com/github/ahatasham5/Breast_Cancer_Prognosis/blob/main/test-vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <div style="color:yellow;display:inline-block;border-radius:5px;background-color:#007BA7;font-family:Nexa;overflow:hidden"><p style="padding:15px;color:yellow;overflow:hidden;font-size:100%;letter-spacing:0.5px;margin:0"><b> </b> Import Modules</p></div>

In [1]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

In [2]:
import torch
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
import torchvision.models as models

def create_model(model_name, num_classes):
    """Creates a model based on the given name.

    Args:
        model_name (str): Name of the model (e.g., 'resnet50', 'vgg16', 'alexnet')
        num_classes (int): Number of output classes for the model

    Returns:
        torch.nn.Module: The initialized model.
    """
    if model_name == 'resnet50':
        model = models.resnet50(pretrained=True)
    elif model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
    elif model_name == 'alexnet':
        model = models.alexnet(pretrained=True)
    else:
        raise ValueError(f"Unknown model name: {model_name}")

    # Freeze pre-trained layers
    for param in model.parameters():
        param.requires_grad = False

    # Replace final fully connected layer
    num_ftrs = model.classifier[6].in_features  # Adjust index if necessary
    model.classifier[6] = nn.Linear(num_ftrs, num_classes)

    return model

In [4]:
# Choose your desired model
model_name = 'vgg16'  # or 'vgg16', 'alexnet'
num_classes = 1  # For binary classification

# Create the model
model = create_model(model_name, num_classes)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move the model to your device (CPU or GPU)

# ... (your training and evaluation code)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:08<00:00, 64.5MB/s]


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Dataset/

/content/drive/MyDrive/Dataset


In [7]:
ls

'Biglycan breast cancer dataset'/   checkpoint.pth   model_checkpoint.pth   model_weights.pth


In [8]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for ResNet
])

# Load your dataset
dataset = datasets.ImageFolder("Biglycan breast cancer dataset", transform=transform)

# Splitting dataset into train and validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
ls

'Biglycan breast cancer dataset'/   checkpoint.pth   model_checkpoint.pth   model_weights.pth


In [13]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.classifier[6].parameters(), lr=5e-5)

In [14]:
ls

'Biglycan breast cancer dataset'/   checkpoint.pth   model_checkpoint.pth   model_weights.pth


In [16]:
import torch

# Define the checkpoint file path
checkpoint_path = 'model_checkpoint_vgg.pth'

num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Save the checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }, checkpoint_path)

Epoch 1, Loss: 0.6904701722992791
Epoch 2, Loss: 0.6921040217081705
Epoch 3, Loss: 0.6932942801051669
Epoch 4, Loss: 0.6776271065076193
Epoch 5, Loss: 0.6822038756476508
Epoch 6, Loss: 0.6863623592588637
Epoch 7, Loss: 0.6534800330797831
Epoch 8, Loss: 0.6631967425346375
Epoch 9, Loss: 0.6505694256888496
Epoch 10, Loss: 0.6642810040050082
Epoch 11, Loss: 0.6431931654612223
Epoch 12, Loss: 0.6478411157925924
Epoch 13, Loss: 0.6530547009574043
Epoch 14, Loss: 0.6295138266351488
Epoch 15, Loss: 0.6314088834656609
Epoch 16, Loss: 0.6469560464223226
Epoch 17, Loss: 0.617464198006524
Epoch 18, Loss: 0.6124784747759501
Epoch 19, Loss: 0.6218240923351712
Epoch 20, Loss: 0.6058646440505981
Epoch 21, Loss: 0.6003322137726678
Epoch 22, Loss: 0.6082426177130805
Epoch 23, Loss: 0.6151795784632365
Epoch 24, Loss: 0.5985756582683988
Epoch 25, Loss: 0.5806763701968722
Epoch 26, Loss: 0.593329906463623
Epoch 27, Loss: 0.5858253306812711
Epoch 28, Loss: 0.5917756425009834
Epoch 29, Loss: 0.5754106442133

In [17]:
ls

'Biglycan breast cancer dataset'/   model_checkpoint.pth       model_weights.pth
 checkpoint.pth                     model_checkpoint_vgg.pth


In [18]:
import torch

# Define the checkpoint file path
checkpoint_path = 'model_checkpoint_vgg.pth'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)
start_epoch = checkpoint['epoch'] + 1
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss = checkpoint['loss']

print(f"Resuming training from epoch {start_epoch}, with loss: {loss}")

num_epochs = 100
for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).type_as(outputs))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Save the checkpoint after each epoch
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss / len(train_loader)
    }, checkpoint_path)

Resuming training from epoch 100, with loss: 0.4891049332088894


In [19]:
import torch

# Load the saved model weights
model.load_state_dict(torch.load('model_.pth'))

RuntimeError: Error(s) in loading state_dict for VGG:
	Missing key(s) in state_dict: "features.0.weight", "features.0.bias", "features.2.weight", "features.2.bias", "features.5.weight", "features.5.bias", "features.7.weight", "features.7.bias", "features.10.weight", "features.10.bias", "features.12.weight", "features.12.bias", "features.14.weight", "features.14.bias", "features.17.weight", "features.17.bias", "features.19.weight", "features.19.bias", "features.21.weight", "features.21.bias", "features.24.weight", "features.24.bias", "features.26.weight", "features.26.bias", "features.28.weight", "features.28.bias", "classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias", "classifier.6.weight", "classifier.6.bias". 
	Unexpected key(s) in state_dict: "conv1.weight", "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var", "bn1.num_batches_tracked", "layer1.0.conv1.weight", "layer1.0.bn1.weight", "layer1.0.bn1.bias", "layer1.0.bn1.running_mean", "lay

In [20]:
import torch

# Load the checkpoint
checkpoint = torch.load('model_checkpoint_vgg.pth')
model.load_state_dict(checkpoint['model_state_dict'])

# Optionally, you can also load the optimizer state and other saved data
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

In [21]:
model.eval()  # Set model to evaluation mode
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.sigmoid(outputs) > 0.5
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(labels.cpu().numpy())

# Calculate and print the evaluation metrics
print("Confusion Matrix:")
print(confusion_matrix(all_targets, np.array(all_preds).flatten()))

print("\nClassification Report:")
print(classification_report(all_targets, np.array(all_preds).flatten(), target_names=['Healthy', 'Cancerous']))


Confusion Matrix:
[[36  8]
 [10 14]]

Classification Report:
              precision    recall  f1-score   support

     Healthy       0.78      0.82      0.80        44
   Cancerous       0.64      0.58      0.61        24

    accuracy                           0.74        68
   macro avg       0.71      0.70      0.70        68
weighted avg       0.73      0.74      0.73        68



In [22]:
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)

        # Apply sigmoid since we used BCEWithLogitsLoss
        predicted_probs = torch.sigmoid(outputs)

        # Convert probabilities to predicted class (0 or 1)
        predicted = predicted_probs > 0.5

        total += labels.size(0)
        correct += (predicted.flatten() == labels).sum().item()

accuracy = correct / total
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 73.53%
